In [2]:
!pip install PyMuPDF
!pip install konlpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 30.3 MB/s eta 0:00:00


In [106]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import io
import fitz
import re
from konlpy.tag import Mecab, Komoran
import pandas as pd

In [151]:
# lab 4-1
# problem 1: 1페이지 상위 5개 리포트 종목명, 날짜, 리포트 pdf 주소를 한 dataframe에 저장

page1_url = 'https://finance.naver.com/research/company_list.naver'
page1_html = urlopen(page1_url)
bs_obj = BeautifulSoup(page1_html, 'html.parser')
page1_reports = bs_obj.find_all("td", {"style":"padding-left:10"})
page1_pdf = bs_obj.find_all("td",{"class":"file"})
page1_date = bs_obj.find_all("td",{"class":"date"})
page1_5_reports = page1_reports[0:5] # 바로바로 업데이트되기 때문에 값이 변경됨
page1_5_pdf = page1_pdf[0:5]
page1_5_date = page1_date[0:10]


# 리포트 내부 값 저장
page1_report_title = []
page1_report_date = []
page1_report_pdf = []

for i in range(5):
  page1_report_title.append(page1_5_reports[i].find("a")['title'])
  page1_report_date.append(page1_5_date[2*i].text)
  page1_report_pdf.append(page1_5_pdf[i].find("a")['href'])

page1_df_5 = pd.DataFrame(
  {  '종목명': page1_report_title,
    '날짜' :page1_report_date,
    'pdf 주소' : page1_report_pdf
  }
)

print(page1_df_5)

      종목명        날짜                                             pdf 주소
0   롯데웰푸드  24.11.04  https://stock.pstatic.net/stock-research/compa...
1    기업은행  24.11.04  https://stock.pstatic.net/stock-research/compa...
2    엘앤에프  24.11.04  https://stock.pstatic.net/stock-research/compa...
3  에코프로비엠  24.11.04  https://stock.pstatic.net/stock-research/compa...
4    산일전기  24.11.04  https://stock.pstatic.net/stock-research/compa...


In [99]:
# problem 2: 1, 2, 3페이지에 있는 모든 리포트들에 대해 종목명, 날짜, 리포트 pdf주소를 한 dataframe에 저장

# 다음페이지의 주소 추출
thispage = bs_obj.find_all("td", {"class":"on"}) # 현재 페이지 <a href="/research/company_list.naver?&amp;page=1">1</a>

thispage_a_href = thispage[0].find("a")['href'] # /research/company_list.naver?&amp;page=1

page2_url = "http://finance.naver.com" + thispage_a_href[:-1]+str(int(thispage_a_href[-1])+1) # page 2
page3_url = "http://finance.naver.com" + thispage_a_href[:-1]+str(int(thispage_a_href[-1])+2) # page 3

def page_parsing(page_url):
  page_html = urlopen(page_url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')
  page_reports = bs_obj.find_all("td", {"style":"padding-left:10"})
  page_pdf = bs_obj.find_all("td",{"class":"file"})
  page_date = bs_obj.find_all("td",{"class":"date"})


  # 리포트 내부 값 저장
  page_report_title = []
  page_report_date = []
  page_report_pdf = []

  for i in range(30):
    page_report_title.append(page_reports[i].find("a")['title'])
    page_report_date.append(page1_date[2*i].text)
    page_report_pdf.append(page1_pdf[i].find("a")['href'])

  page_df = pd.DataFrame(
    {  '종목명': page_report_title,
      '날짜' :page_report_date,
      'pdf 주소' : page_report_pdf
    }
  )

  return page_df


page1_df = page_parsing(page1_url)
page2_df = page_parsing(page2_url)
page3_df = page_parsing(page3_url)
result_df = pd.concat([page1_df, page2_df, page3_df])
print(result_df)

          종목명        날짜                                             pdf 주소
0       롯데웰푸드  24.11.04  https://stock.pstatic.net/stock-research/compa...
1        기업은행  24.11.04  https://stock.pstatic.net/stock-research/compa...
2        엘앤에프  24.11.04  https://stock.pstatic.net/stock-research/compa...
3      에코프로비엠  24.11.04  https://stock.pstatic.net/stock-research/compa...
4        산일전기  24.11.04  https://stock.pstatic.net/stock-research/compa...
..        ...       ...                                                ...
25     현대글로비스  24.11.04  https://stock.pstatic.net/stock-research/compa...
26      DL이앤씨  24.11.04  https://stock.pstatic.net/stock-research/compa...
27       GS건설  24.11.04  https://stock.pstatic.net/stock-research/compa...
28       SOOP  24.11.04  https://stock.pstatic.net/stock-research/compa...
29  한화에어로스페이스  24.11.04  https://stock.pstatic.net/stock-research/compa...

[90 rows x 3 columns]


In [131]:
# problem 3: 1, 2, 3페이지에 있는 모든 리포트들에 대해 종목명, 날짜, 제목, 본문을 한 dataframe에 저장

# 본문 저장하는 거 추가하기

# page_html=urlopen('https://finance.naver.com/research/company_read.naver?nid=78281&page=1')
# bs_obj2 = BeautifulSoup(page_html, 'html.parser')
# page_content = bs_obj2.find_all("p")
# print(page_content[5]) # 5번째가 전체 p 태그중에서 본문의미함

def pages_parsing(page_url, inner_content_links):
  page_html = urlopen(page_url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')
  page_reports = bs_obj.find_all("td", {"style":"padding-left:10"})
  page_pdf = bs_obj.find_all("td",{"class":"file"})
  page_date = bs_obj.find_all("td",{"class":"date"})

  # 리포트 내부 값 저장
  page_report_title = []
  page_report_date = []

  for i in range(30):
    page_report_title.append(page_reports[i].find("a")['title'])
    page_report_date.append(page_date[2*i].text)

  page_df = pd.DataFrame(
    {  '종목명': page_report_title,
      '날짜' :page_report_date,

    }
    )

  page_report_title = []
  page_report_content = []

  for link in inner_content_links:
    # 각 링크마다 접속
    page_link = urlopen(link)
    parser = BeautifulSoup(page_link, 'html.parser')

    # 본문 크롤링해서 가져오기
    try:
      title = parser.find("td", {'class':'view_cnt'}).find_all('p')[0].get_text(strip=True)
      content = parser.find("td", {'class':'view_cnt'}).find_all('p')[1].get_text(strip=True)
    except:
      pass
    page_report_title.append(title)
    page_report_content.append(content)
    # print(content)

  page_df['제목'] = page_report_title
  page_df['본문'] = page_report_content

  return page_df


def inner_content(url):
  # url : 메인 페이지(1,2,3 ...)
  # 위에서부터 본문 주소를 다 불러오기
  page_html = urlopen(url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')

  page_reports = bs_obj.find_all("td")
  # print(page_reports)

  content_links =  ['https://finance.naver.com/research/'+a['href'] for a in bs_obj.find_all('a') if a['href'].startswith('company_read')]


  # print(content_links)
  return content_links

#__main__

page1_content_link = inner_content(page1_url) # 본문 주소 받아오기
page2_content_link = inner_content(page2_url)
page3_content_link = inner_content(page3_url)

# 순서대로 본문 불러오기
page1_df_3 = page_parsing(page1_url, page1_content_link)
page2_df_3 = page_parsing(page2_url, page2_content_link)
page3_df_3 = page_parsing(page3_url, page3_content_link)

# result dataframe
result3_frame = pd.concat([page1_df_3, page2_df_3, page3_df_3])

print(result3_frame)


          종목명        날짜                                        제목  \
0       롯데웰푸드  24.11.04  3Q24Re: 국내외 아쉬운 외형 성장과 카카오 원가 부담으로 수익 감소   
1        기업은행  24.11.04     3Q24 순이익 8,014억원(+10.0% YoY)으로 기대치 상회   
2        엘앤에프  24.11.04   3Q24 Re: 고객사 재고조정과 유럽 수요회복 지연으로 영업적자 지속   
3      에코프로비엠  24.11.04  3Q24P Re: 재고평가손실 포함 영업적자 412억원으로 컨센서스 하회   
4        산일전기  24.11.04              24.3Q Review: 영업이익률 34.9% 달성   
..        ...       ...                                       ...   
25     현대글로비스  24.11.01              실적도 좋은데, 주주환원까지 좋으니 목표주가를 상향   
26      DL이앤씨  24.11.01                        3Q24 잠정실적: 컨센서스 상회   
27       GS건설  24.11.01                        3Q24 잠정실적: 컨센서스 부합   
28       SOOP  24.11.01                 3Q24 Re: 일회성 비용으로 컨센서스 하회   
29  한화에어로스페이스  24.11.01                    3Q24 Re : 상상을 뛰어넘은 호실적   

                                                   본문  
0   롯데웰푸드의 3분기 실적은 연결기준 매출액 1조 785억원(-0.7% YoY), 영...  
1   3Q24 지배 순이익은 8,014억원(+10.0% YoY)으로 시장 기대치를 +9....  
2  

In [163]:
'''
problem 4 : 1페이지에 있는 모든 리포트들에 대해 제목과 본문을
Komoran으로 분석한 뒤 형태소의 종류별 개수를 구하라
'''
from collections import Counter

page1_df_reports = page1_df_3[['제목', '본문']]
# print(page1_df_reports)

komoran = Komoran()
pos_counts = []

for content in page1_df_reports['제목']:
  # print(content)
  pos = komoran.pos(content)
  pos_counter = Counter(tag for _, tag in pos)
  pos_counts.append(pos_counter)

for content in page1_df_reports['본문']:
  # print(content)
  pos = komoran.pos(content)
  pos_counter = Counter(tag for _, tag in pos)
  pos_counts.append(pos_counter)

pos_df = pd.DataFrame(pos_counts).fillna(0)
page1_df = page1_df.join(pos_df)
print(page1_df)

          종목명        날짜                                             pdf 주소  \
0       롯데웰푸드  24.11.04  https://stock.pstatic.net/stock-research/compa...   
1        기업은행  24.11.04  https://stock.pstatic.net/stock-research/compa...   
2        엘앤에프  24.11.04  https://stock.pstatic.net/stock-research/compa...   
3      에코프로비엠  24.11.04  https://stock.pstatic.net/stock-research/compa...   
4        산일전기  24.11.04  https://stock.pstatic.net/stock-research/compa...   
5        기업은행  24.11.04  https://stock.pstatic.net/stock-research/compa...   
6       피에스케이  24.11.04  https://stock.pstatic.net/stock-research/compa...   
7       DL이앤씨  24.11.04  https://stock.pstatic.net/stock-research/compa...   
8      에코프로비엠  24.11.04  https://stock.pstatic.net/stock-research/compa...   
9       롯데웰푸드  24.11.04  https://stock.pstatic.net/stock-research/compa...   
10       호텔신라  24.11.04  https://stock.pstatic.net/stock-research/compa...   
11        네오팜  24.11.04  https://stock.pstatic.net/stock-researc

In [179]:
'''
problem 5: 1페이지에 있는 모든 리포트들에 대해 pdf 파일을 Komoran으로
분석한 뒤, 형태소의 종류별 개수를 구하라(종목명, 날짜, 제목, 본문, 명사개수 등)한 df에 저장
'''
from collections import Counter


# 내부 링크 들어가서 content 가져오기
def pdf_pos(pdf):
  pos_counts = []

  for link in url:
    response = requests.get(link)
    pdf = io.BytesIO(response.content)

    # pdf to text
    text_by_page = []
    with fitz.open(stream=pdf) as doc:
      for page in doc:
        text_by_page.append(page.get_text())

    sentence = re.split('\n', text_by_page[0].strip())

    text_by_page_enter = text_by_page[0].replace('\n',' ')
    komoran = Komoran()
    pos = komoran.pos(text_by_page_enter)

    pos_counter = Counter(tag for _, tag in pos)
    pos_counts.append(pos_counter)
    morph = pd.DataFrame(pos).groupby(1)[1].count()
    pos_df = pd.DataFrame(pos_counts).fillna(0)
  return pos_df


#__main__
url = page1_df['pdf 주소']

pos_df = pdf_pos(url)
page1_df_5 = page1_df_3.join(pos_df, lsuffix='_caller', rsuffix='_other')
print(page1_df_5)

          종목명        날짜                                                 제목  \
0       롯데웰푸드  24.11.04           3Q24Re: 국내외 아쉬운 외형 성장과 카카오 원가 부담으로 수익 감소   
1        기업은행  24.11.04              3Q24 순이익 8,014억원(+10.0% YoY)으로 기대치 상회   
2        엘앤에프  24.11.04            3Q24 Re: 고객사 재고조정과 유럽 수요회복 지연으로 영업적자 지속   
3      에코프로비엠  24.11.04           3Q24P Re: 재고평가손실 포함 영업적자 412억원으로 컨센서스 하회   
4        산일전기  24.11.04                       24.3Q Review: 영업이익률 34.9% 달성   
5        기업은행  24.11.04  2024년 3분기 연결당기순이익 8,014억원, QoQ 31.8%↑, YoY 10.0%↑   
6       피에스케이  24.11.04                          3Q24 Preview : 컨센서스 상회 전망   
7       DL이앤씨  24.11.04                       투자의견 ‘매수’ 및 목표주가 39,000원 유지.   
8      에코프로비엠  24.11.04                        3Q24 Review 컨센하회, 지속되는 업황부진   
9       롯데웰푸드  24.11.04                      3Q24 Re: 코코아 원가 부담으로 시장기대치 하회   
10       호텔신라  24.11.04                   3분기 영업적자 170억원(YoY -247억원, 적자전환)   
11        네오팜  24.11.04                           3Q24 Review: 컨

In [ ]:
'''
problem 6 : 101-120 페이지에 있는 모든 리포트들에 대해 제목과 본문을 Komoran으로
분석한 뒤, 형태소의 종류별 개수를 구하라(종목명, 날짜, 제목, 본문, 명사개수 등)한 df에 저장
'''
# 100-120페이지 주소 가져오기       -(1)

# komoran 분석
# 형태소 분석


# 제목 + 본문 df에 저장하기        -(2)

def pages_parsing(page_url, inner_content_links):
  page_html = urlopen(page_url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')
  page_reports = bs_obj.find_all("td", {"style":"padding-left:10"})
  page_pdf = bs_obj.find_all("td",{"class":"file"})
  page_date = bs_obj.find_all("td",{"class":"date"})

  # 리포트 내부 값 저장
  page_report_title = []
  page_report_date = []

  for i in range(30):
    page_report_title.append(page_reports[i].find("a")['title'])
    page_report_date.append(page_date[2*i].text)

  page_df = pd.DataFrame(
    {  '종목명': page_report_title,
      '날짜' :page_report_date,

    }
    )

  page_report_title = []
  page_report_content = []

  for link in inner_content_links:
    # 각 링크마다 접속
    page_link = urlopen(link)
    parser = BeautifulSoup(page_link, 'html.parser')

    # 본문 크롤링해서 가져오기
    try:
      title = parser.find("td", {'class':'view_cnt'}).find_all('p')[0].get_text(strip=True)
      content = parser.find("td", {'class':'view_cnt'}).find_all('p')[1].get_text(strip=True)

    except:
      title = None
      content = None
    page_report_title.append(title)
    page_report_content.append(content)
    # print(content)

  page_df['제목'] = page_report_title
  page_df['본문'] = page_report_content

  return page_df


def inner_content(url):
  # url : 메인 페이지(1,2,3 ...)
  # 위에서부터 본문 주소를 다 불러오기
  page_html = urlopen(url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')

  page_reports = bs_obj.find_all("td")
  # print(page_reports)

  content_links =  ['https://finance.naver.com/research/'+a['href'] for a in bs_obj.find_all('a') if a['href'].startswith('company_read')]

  print(content_links)
  return content_links


# __main__

from collections import Counter

page101_url = 'https://finance.naver.com/research/company_list.naver'

page_list = [] # 101-120p url
for i in range(1,21):
  page_list.append(page101_url+'?&page='+str(i+100))

result_6_df = pd.DataFrame()

for page in page_list:
  print(i)
  page_content_link = inner_content(page)
  page_df = pages_parsing(page, page_content_link)
  result_6_df = pd.concat([result_6_df, page_df])
  print(result_6_df.shape)
  i+=1



In [168]:

# ==========================================================
komoran = Komoran()
pos_counts6 = []


for content in result_6_df['본문']:
  pos = komoran.pos(str(content))
  pos_counter = Counter(tag for _, tag in pos)
  pos_counts6.append(pos_counter)

result_6_pos = pd.DataFrame(pos_counts6).fillna(0)
# Specify lsuffix and rsuffix to differentiate overlapping columns
result_6_df = result_6_df.join(result_6_pos, lsuffix='_caller', rsuffix='_other')
print(result_6_df)

          종목명        날짜                              제목  \
0      현대오토에버  24.07.16          QoQ 반등과 함께 하반기 성장성에 주목   
1         이노션  24.07.16               2분기 영업이익 357억원 전망   
2       파라다이스  24.07.16   2분기 영업이익 409억원(yoy -25.4%) 전망   
3     삼성에스디에스  24.07.16  2024 안정적 실적 전망 + AI 신성장 동력에 주목   
4      한국항공우주  24.07.16      2분기 영업이익 512억원, 컨센서스 하회 전망   
..        ...       ...                             ...   
25       한미약품  24.06.19                            None   
26  오픈엣지테크놀로지  24.06.19                            기업개요   
27        오리온  24.06.19  소외된 K-푸드 대장, 해외 매출 성장률 반등이 Key   
28        아바코  24.06.19                I. IR 後記 & 실적 전망   
29      대한광통신  24.06.19           인공지능의 확산, 광섬유 소비도 늘린다   

                                                   본문  SN_caller  SL_caller  \
0   1Q24 실적 발표 이후 2Q24 반등 타이밍 진입. QoQ 계단식 실적 성장 전망...        5.0        5.0   
1   2분기 매출총이익 2,190억원(yoy +2.6%), 영업이익 357억원(yoy -...       21.0        9.0   
2   매출액 2,822억원(yoy +2.5%), 영업이익 409억원(yoy -25.4%,... 